In [125]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import BorderlineSMOTE
from data_process import deal_data
import pandas as pd
from imblearn.over_sampling import ADASYN

In [126]:
bc = deal_data()
X = bc.drop(['pCR (outcome)', 'RelapseFreeSurvival (outcome)', 'ID'], axis=1)
y = bc['pCR (outcome)']
print(X.shape)

(395, 117)


In [129]:
print('Original dataset shape %s' % Counter(y))
sm = BorderlineSMOTE(random_state=1,kind="borderline-1")
new_X, new_y = sm.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(new_y))

Original dataset shape Counter({0: 299, 1: 96})
Resampled dataset shape Counter({1: 299, 0: 299})


In [130]:
kf = KFold(n_splits=5, shuffle=True, random_state=10)
train_predict_score = []
test_predict_score = []
for train_index, test_index in kf.split(new_X, new_y):
    each_x_train, each_x_test = new_X.take(train_index, axis=0), new_X.take(test_index, axis=0)
    each_y_train, each_y_test = new_y.take(train_index, axis=0), new_y.take(test_index, axis=0)
    mlp = MLPClassifier(hidden_layer_sizes=(10, 10), activation='logistic', solver='sgd', learning_rate_init=0.1, max_iter=1000)
    mlp.fit(each_x_train, each_y_train)
    each_y_train_predict = mlp.predict(each_x_train)
    each_y_test_predict = mlp.predict(each_x_test)
    train_predict_score.append(accuracy_score(each_y_train, each_y_train_predict))
    test_predict_score.append(accuracy_score(each_y_test, each_y_test_predict))

In [131]:
print('train score:', np.mean(train_predict_score))
print('test score:', np.mean(test_predict_score))

train score: 0.5016727666599697
test score: 0.505
